In [8]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.signal import convolve2d, convolve, gaussian

def fastkde(x, y, gridsize=(200, 200), extents=None, nocorrelation=False, weights=None, adjust=1.):
    """
    A fft-based Gaussian kernel density estimate (KDE)
    for computing the KDE on a regular grid
    Note that this is a different use case than scipy's original
    scipy.stats.kde.gaussian_kde
    IMPLEMENTATION
    --------------
    Performs a gaussian kernel density estimate over a regular grid using a
    convolution of the gaussian kernel with a 2D histogram of the data.
    It computes the sparse bi-dimensional histogram of two data samples where
    *x*, and *y* are 1-D sequences of the same length. If *weights* is None
    (default), this is a histogram of the number of occurences of the
    observations at (x[i], y[i]).
    histogram of the data is a faster implementation than numpy.histogram as it
    avoids intermediate copies and excessive memory usage!
    This function is typically *several orders of magnitude faster* than
    scipy.stats.kde.gaussian_kde.  For large (>1e7) numbers of points, it
    produces an essentially identical result.
    Boundary conditions on the data is corrected by using a symmetric /
    reflection condition. Hence the limits of the dataset does not affect the
    pdf estimate.
    INPUTS
    ------
        x, y:  ndarray[ndim=1]
            The x-coords, y-coords of the input data points respectively
        gridsize: tuple
            A (nx,ny) tuple of the size of the output grid (default: 200x200)
        extents: (xmin, xmax, ymin, ymax) tuple
            tuple of the extents of output grid (default: extent of input data)
        nocorrelation: bool
            If True, the correlation between the x and y coords will be ignored
            when preforming the KDE. (default: False)
        weights: ndarray[ndim=1]
            An array of the same shape as x & y that weights each sample (x_i,
            y_i) by each value in weights (w_i).  Defaults to an array of ones
            the same size as x & y. (default: None)
        adjust : float
            An adjustment factor for the bw. Bandwidth becomes bw * adjust.
    OUTPUTS
    -------
        g: ndarray[ndim=2]
            A gridded 2D kernel density estimate of the input points.
        e: (xmin, xmax, ymin, ymax) tuple
            Extents of g
    """
    # Variable check
    x, y = np.asarray(x), np.asarray(y)
    x, y = np.squeeze(x), np.squeeze(y)

    if x.size != y.size:
        raise ValueError('Input x & y arrays must be the same size!')

    n = x.size

    if weights is None:
        # Default: Weight all points equally
        weights = np.ones(n)
    else:
        weights = np.squeeze(np.asarray(weights))
        if weights.size != x.size:
            raise ValueError('Input weights must be an array of the same size as input x & y arrays!')

    # Optimize gridsize ------------------------------------------------------
    #Make grid and discretize the data and round it to the next power of 2
    # to optimize with the fft usage
    if gridsize is None:
        gridsize = np.asarray([np.max((len(x), 512.)), np.max((len(y), 512.))])
    gridsize = 2 ** np.ceil(np.log2(gridsize))  # round to next power of 2
    print(f"Gridsize: {gridsize}")
    
    nx, ny = gridsize

    # Make the sparse 2d-histogram -------------------------------------------
    # Default extents are the extent of the data
    if extents is None:
        xmin, xmax = x.min(), x.max()
        ymin, ymax = y.min(), y.max()
    else:
        xmin, xmax, ymin, ymax = map(float, extents)
    dx = (xmax - xmin) / (nx - 1)
    dy = (ymax - ymin) / (ny - 1)
    print(f"xMin: {xmin}, xMax: {xmax}, yMin: {ymin}, yMax: {ymax}")
    
    
    # Basically, this is just doing what np.digitize does with one less copy
    # xyi contains the bins of each point as a 2d array [(xi,yi)]
    xyi = np.vstack((x,y)).T
    xyi -= [xmin, ymin]
    xyi /= [dx, dy]
    xyi = np.floor(xyi, xyi).T
    print(f"bins: \n{xyi}")

    # Next, make a 2D histogram of x & y.
    # Exploit a sparse coo_matrix avoiding np.histogram2d due to excessive
    # memory usage with many points
    grid = coo_matrix((weights, xyi), shape=(nx, ny)).toarray()

    # Kernel Preliminary Calculations ---------------------------------------
    # Calculate the covariance matrix (in pixel coords)
    cov = np.cov(xyi)

    if nocorrelation:
        cov[1,0] = 0
        cov[0,1] = 0
        
    print(f"Covariance: \n{cov}")

    # Scaling factor for bandwidth
    scotts_factor = n ** (-1.0 / 6.) * adjust  # For 2D
    print(f"Scotts Factor: {scotts_factor}")

    # Make the gaussian kernel ---------------------------------------------

    # First, determine the bandwidth using Scott's rule
    # (note that Silvermann's rule gives the # same value for 2d datasets)
    std_devs = np.diag(np.sqrt(cov))
    kern_nx, kern_ny = np.round(scotts_factor * 2 * np.pi * std_devs)

    # Determine the bandwidth to use for the gaussian kernel
    inv_cov = np.linalg.inv(cov * scotts_factor ** 2)

    # x & y (pixel) coords of the kernel grid, with <x,y> = <0,0> in center
    xx = np.arange(kern_nx, dtype=np.float) - kern_nx / 2.0
    yy = np.arange(kern_ny, dtype=np.float) - kern_ny / 2.0
    xx, yy = np.meshgrid(xx, yy)

    # Then evaluate the gaussian function on the kernel grid
    kernel = np.vstack((xx.flatten(), yy.flatten()))
    kernel = np.dot(inv_cov, kernel) * kernel
    kernel = np.sum(kernel, axis=0) / 2.0
    kernel = np.exp(-kernel)
    kernel = kernel.reshape((int(kern_ny), int(kern_nx)))
    
    print(f"Standard Deviation: {std_devs}")
    print(f"Kern_nx: {kern_nx}")
    print(f"Kern_ny: {kern_ny}")
    print(f"Bandwidth: \n{inv_cov}")
    print(f"Meshgrid: {xx.shape}, {yy.shape}")
    print(f"Kernel \n{kernel.shape}")

    #---- Produce the kernel density estimate --------------------------------

    # Convolve the histogram with the gaussian kernel
    # use boundary=symm to correct for data boundaries in the kde
    grid = convolve2d(grid, kernel, mode='same', boundary='symm')

    # Normalization factor to divide result by so that units are in the same
    # units as scipy.stats.kde.gaussian_kde's output.
    norm_factor = 2 * np.pi * cov * scotts_factor ** 2
    norm_factor = np.linalg.det(norm_factor)
    norm_factor = n * dx * dy * np.sqrt(norm_factor)

    # Normalize the result
    grid /= norm_factor
    
    print(f"Grid Before Norm: \n{grid.shape}")
    print(f"Normalization Factor: {norm_factor}")
    print(f"Grid After Norm: \n{grid}")
    print(f"Grid Shape: {grid.shape}")

In [9]:
x = np.array([0.3395,    0.9783,    0.3895,    0.7022,    0.0185,    0.1849,    0.6926,    0.8179,    0.3845,    0.5245])
y = np.array([0.7747,    0.7053,    0.5655,    0.6056,    0.1807,    0.0696,    0.1127,    0.4404,    0.6015,    0.6682])

In [10]:
fastkde(x,y)

Gridsize: [256. 256.]
xMin: 0.0185, xMax: 0.9783, yMin: 0.0696, yMax: 0.7747
bins: 
[[ 85. 255.  98. 181.   0.  44. 179. 212.  97. 134.]
 [255. 229. 179. 193.  40.   0.  15. 134. 192. 216.]]
Covariance: 
[[6197.61111111 2721.72222222]
 [2721.72222222 8770.67777778]]
Scotts Factor: 0.6812920690579612
Standard Deviation: [78.72490782 93.65189682]
Kern_nx: 337.0
Kern_ny: 401.0
Bandwidth: 
[[ 0.00040247 -0.0001249 ]
 [-0.0001249   0.0002844 ]]
Meshgrid: (401, 337), (401, 337)
Kernel 
(401, 337)
Grid Before Norm: 
(256, 256)
Normalization Factor: 2.0797590333767157
Grid After Norm: 
[[2.853681   2.85331732 2.85222652 ... 1.33934187 1.33982924 1.34012175]
 [2.85343446 2.8531905  2.85221945 ... 1.33970436 1.34015225 1.34040537]
 [2.85269498 2.85257075 2.85171951 ... 1.34055473 1.34096332 1.34117715]
 ...
 [0.85604241 0.85620973 0.856251   ... 2.54984066 2.55036483 2.550547  ]
 [0.85569311 0.85578397 0.8557491  ... 2.54983087 2.55046531 2.55075774]
 [0.85548327 0.8554974  0.85538602 ... 2.5496